Demonstration of the framework from https://github.com/SSobirov/English_to_Russian-TTS-MelGAN


Converted Samples are located in GitHub repository

If you have any problems with running this demo please contanct me on my email address.
zy22565@nottingham.edu.cn or shohinzya@gmail.com

In [1]:
# install minimal components
!pip install -q PyYaml parallel-wavegan unidecode ConfigArgparse  nltk
!git clone -q https://github.com/espnet/espnet.git
!cd espnet && git fetch && git checkout -b v.0.6.1 1e8b6ce88d57b53d1b60cbb3f306652468b0ab63

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 245kB 9.9MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 1.6MB 54.6MB/s 
     |████████████████████████████████| 204kB 45.4MB/s 
Switched to a new branch 'v.0.6.1'


Downloading pretrained MelGAN

In [3]:
# download pretrained model
import os
if not os.path.exists("downloads/en/melgan"):
    !./espnet/utils/download_from_google_drive.sh \
        https://drive.google.com/open?id=1_a8faVA5OGCzIcJNw4blQYjfG4oA9VEt downloads/en/melgan tar.gz

# set path
vocoder_path = "downloads/en/melgan/train_nodev_ljspeech_melgan.v3.long/checkpoint-4000000steps.pkl"
vocoder_conf = "downloads/en/melgan/train_nodev_ljspeech_melgan.v3.long/config.yml"

print("Sucessfully finished download.")

--2020-05-18 04:40:22--  https://drive.google.com/uc?export=download&id=1_a8faVA5OGCzIcJNw4blQYjfG4oA9VEt
Resolving drive.google.com (drive.google.com)... 74.125.202.139, 74.125.202.101, 74.125.202.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.202.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘downloads/en/melgan/pdxm5x.tar.gz’

downloads/en/melgan     [ <=>                ]   3.20K  --.-KB/s    in 0s      

2020-05-18 04:40:22 (44.7 MB/s) - ‘downloads/en/melgan/pdxm5x.tar.gz’ saved [3277]


gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3277    0  3277    0     0  15241      0 --:--:-- --:--:-- --:--:-- 15171
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Cur

Upload finetuned folder from the provided project folder and move it to /content/downloads/en/ and then run the codes below. The resulting structure must be as following: /content/downloads/en/finetuned/


In [0]:
trans_type = "char"
dict_path = "/content/downloads/en/finetuned/data/lang_1char/train_units.txt"
model_path = "/content/downloads/en/finetuned/exp/train_pytorch_train_pytorch_transformer+spkemb/results/model.loss.best"

setup

In [31]:
# add path
import sys
sys.path.append("espnet/egs/ljspeech/tts1/local")
sys.path.append("espnet")

# define device
import torch
device = torch.device("cuda")

# define E2E-TTS model
from argparse import Namespace
from espnet.asr.asr_utils import get_model_conf
from espnet.asr.asr_utils import torch_load
from espnet.utils.dynamic_import import dynamic_import
idim, odim, train_args = get_model_conf(model_path)
model_class = dynamic_import(train_args.model_module)
model = model_class(idim, odim, train_args)
torch_load(model_path, model)
model = model.eval().to(device)
inference_args = Namespace(**{
    "threshold": 0.5,"minlenratio": 0.0, "maxlenratio": 10.0,
    "use_attention_constraint": True,
    "backward_window": 1, "forward_window":3,
    })

# define neural vocoder
import yaml
import parallel_wavegan.models
with open(vocoder_conf) as f:
    config = yaml.load(f, Loader=yaml.Loader)
vocoder_class = config.get("generator_type", "MelGANGenerator")
vocoder = getattr(parallel_wavegan.models, vocoder_class)(**config["generator_params"])
vocoder.load_state_dict(torch.load(vocoder_path, map_location="cpu")["model"]["generator"])
vocoder.remove_weight_norm()
vocoder = vocoder.eval().to(device)

# define text frontend
from text.cleaners import custom_english_cleaners
with open(dict_path) as f:
    lines = f.readlines()
lines = [line.replace("\n", "").split(" ") for line in lines]
char_to_id = {c: int(i) for c, i in lines}
def frontend(text):
    """Clean text and then convert to id sequence."""
    text = custom_english_cleaners(text)
    
    print(f"Cleaned text: {text}")
    charseq = list(text)
    idseq = []
    for c in charseq:
        if c.isspace():
            idseq += [char_to_id["<space>"]]
        elif c not in char_to_id.keys():
            idseq += [char_to_id["<unk>"]]
        else:
            idseq += [char_to_id[c]]
    idseq += [idim - 1]  # <eos>
    return torch.LongTensor(idseq).view(-1).to(device)

print("Now ready to synthesize!")

Now ready to synthesize!


Upload "copy-vector" file from "tools" folder and run the code below

In [0]:
!chmod +x /content/copy-vector

In [25]:
!/content/copy-vector ark:/content/downloads/en/finetuned/exp/train_pytorch_train_pytorch_transformer+spkemb/spk_xvector.ark ark,t:- >xvector.txt

/content/copy-vector ark:/content/downloads/en/finetuned/exp/train_pytorch_train_pytorch_transformer+spkemb/spk_xvector.ark ark,t:- 
LOG (copy-vector[5.5.689~1-2c7e7]:main():copy-vector.cc:90) Copied 1 vectors.


Important! the "xvector.txt" file is generated in /content folder. Please delete "SEF1" and "[" and "]" character from inside of the file, so that only vectors are there. You will get corresponding error if these three characters are not deleted from the file.


In [0]:
import numpy as np

In [0]:
arr = np.loadtxt("/content/xvector.txt")
xvector = torch.from_numpy(arr).to(device='cuda', dtype=torch.float32)

In [35]:
import time
from scipy.io.wavfile import write
print("Input your favorite sentence in Russian!")
input_text = input()
pad_fn = torch.nn.ReplicationPad1d(2)
with torch.no_grad():
    start = time.time()
    x = frontend(input_text)
    c, _, _ = model.inference(x, inference_args, xvector)
    c = pad_fn(c.unsqueeze(0).transpose(2, 1)).to(device)
    xx = (c,)
    y = vocoder(*xx).view(-1)
rtf = (time.time() - start)
print(f"Waveform generation time = {rtf:5f} seconds")

from IPython.display import display, Audio
write("sample.wav", 20800, y.view(-1).cpu().numpy())
display(Audio(y.view(-1).cpu().numpy(), rate=20800))

Input your favorite sentence in Russian!
I RUKA SULAMIFI POKOILAS' NA EGO GOLOVE, GLADILA EGO VOLNISTYE CHERNYE VOLOSY.
Cleaned text: I RUKA SULAMIFI POKOILAS' NA EGO GOLOVE, GLADILA EGO VOLNISTYE CHERNYE VOLOSY.
Waveform generation time = 1.179695 seconds
